In [1]:
import re

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses,metrics,callbacks

from typing import List
from json import load,dump

from threading import Thread

In [2]:
with open("../data/example_train.json","r") as build_file:
    build_config = eval(build_file.read())
    
with open("../data/example_code.py","r") as code_file:
    code = code_file.read()

In [76]:
def execute_code(_code:str)->None:
    exec(_code)
    globals().update(locals())

In [77]:
class TfGui(keras.callbacks.Callback):
    batch = None 
    epoch = 0
    
    batch_size=8
    batches = 1
    epochs = 1
    
    trainer = None
    
    def __repr__(self,):
        return f"""TfGui(
    batch={self.batch},
    logs={self.logs},
    batch_size={self.batch_size},
    epochs={self.epochs},
)
"""
    
    @property
    def status(self,):
        return {
            "epochs":self.epochs,
            "batchs":self.batches
        }
    
    def __init__(self,trainer):
        self.trainer = trainer
    
    def on_train_begin(self,epoch,logs=None):
        pass
    
    def on_batch_end(self,batch,logs=None):
        self.batch = batch
        self._epoch['log'] = {
            "batch":batch,
            "output":logs
        }
        
    def on_epoch_begin(self,epoch,logs=None):
        self.epoch = epoch
        self._epoch = {
            "epoch":epoch,
            "log":{
                "batch":0,
                "output":None
            },
            "train":{
                "epochs":self.epochs,
                "batches":self.batches
            }
        }
        
        self.trainer.update_log(
            log_type = "epoch",
            log=self._epoch
        )

In [78]:
class Trainer(object):
    build_config = {}
    model = keras.Model
    update_id = 0
    
    logs = []

    def __init__(self):
        globals()['tfgui'] = TfGui(self,)
        self.tfgui = globals()['tfgui']
    
    def update_log(self,log_type:str,log):
        self.logs.append({
            "type":log_type,
            "data":log
        })
        
    def start(self,build_config:dict,code:str)->None:
        
        self.update_log("notif",{"message":"Copiling Code"})
        charset = 'a-zA-Z0-9 .\(\)\{\{\[\] \n=\-_\+,\'\"'
        self.update_log("notif",{"message":"Performing imports"})
        imports, = re.findall("""#import[a-zA-Z\n .,_\-]+#end-import""",code)
        execute_code(imports)
        execute_code(build_config['train_config']['dataset']['value'])
        
        self.update_log("notif",{"message":"Building Model"})
        for level in build_config['levels']:
            for layer in level:
                _code = re.findall(f"""{layer} =[{charset}]+#end-{layer}""",code)
                if len(_code):
                    execute_code(_code[0])

        if build_config['train_config']['optimizer']:
            execute_code(build_config['train_config']['optimizer']['value'])

        if build_config['train_config']['callbacks']:
            for callback in build_config['train_config']['callbacks']:
                execute_code(callback['value'])    
        
        if build_config['train_config']['loss']:
            execute_code(build_config['train_config']['loss']['value'])
            
        self.update_log("notif",{"message":"Compiling Model"})

        comp = build_config['train_config']['compile']['id']
        model = build_config['train_config']['model']['id']
        execute_code(re.findall(f"""{model}.compile[{charset}]+#end-{comp}""",code)[0])
        
        self.tfgui.epochs = int(build_config['train_config']['train']['arguments']['epochs']['value'])
        self.tfgui.batch_size = int(build_config['train_config']['train']['arguments']['batch_size']['value'])
        self.tfgui.batches = np.ceil(globals()[build_config['train_config']['dataset']['id']].train_x.__len__() / tfgui.batch_size).astype(int)
        
        train_code, = re.findall(f"""{model}.fit\([a-z\n\t =_0-9.,\[\]\)]+#end-train_\d+""",code)
        train_thread = Thread(target=execute_code,args=(train_code,))
        train_thread.start()
        self.update_log("notif",{"message":"Training Started"})
    
trainer = Trainer()

In [79]:
trainer.start(build_config,code)

ValueError: Please provide to Input either a `shape` or a `tensor` argument. Note that `shape` does not include the batch dimension.